In [16]:
import parselmouth

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
import pandas as pd
import scipy.signal as signal
import scipy
from curlyBrace import curlyBrace

In [17]:
def draw_pitch(pitch):
    # Extract selected pitch contour, and
    # replace unvoiced samples by NaN to not plot
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    plt.plot(pitch.xs(), pitch_values,color="black")
    plt.grid(True)
    plt.ylim(np.nanmin(pitch_values)-10, np.nanmax(pitch_values)+10)
    plt.ylabel("fundamental frequency [Hz]")
    return pitch_values

In [18]:
def shift(points,mins,maxs):
    mins = pd.Series(mins)
    maxs = pd.Series(maxs)
    for i in range(len(points)):
        min_subs = mins.sub(points[i][1])
        potential = min_subs[min_subs<0].abs()
        if len(potential)==0:
            continue
        new = mins[potential.idxmin()]
        if abs(points[i][1]-new)<0.1:
            points[i][1] = new
        
        max_subs = maxs.sub(points[i][0])
        potential = max_subs[max_subs<0].abs()
        if len(potential)==0:
            continue
        new = maxs[potential.idxmin()]
        if abs(points[i][0]-new)<0.1:
            points[i][0] = new
        #print(min_subs, max_subs)
    return points


def compute_intersection(xs,interp,rolling):
    points = []
    first_roll = rolling.bfill().iloc[0]
    down = first_roll > interp.loc[rolling[rolling==first_roll].index[0]]
    for i in zip(xs,interp,rolling):
        if np.nan in i:
            continue
        curdown = i[1]<=i[2]
        if curdown!=down and curdown==True:
            down = curdown
            points.append([i[0]])
        if curdown!=down and curdown==False:
            down = curdown
            if len(points)!=0:
                points[-1].append(i[0])

    idx = 0
    thresh = 0.1
    while idx<len(points):
        if points[idx][1]-points[idx][0] < thresh:
            points.pop(idx)
        else:
            idx+=1
            
    if len(points)==0:
        return []
        
    if len(points[-1])==0:
        points[-1].append(snd.xmax)
    
    return points



In [19]:
def prune(points,pitch,xs):
    idx = 0
    while idx<len(points):
        val = pitch[np.where(xs==points[idx][1])[0]]
        if np.isnan(val):
            points.pop(idx)
        else:
            idx+=1
    return points

In [23]:
%%time
def makeanim(sound):
    def update_line(num, line):
        i = X_VALS[num]
        line.set_data( [i, i], [0, 600])
        return line, 
    
    snd = parselmouth.Sound(sound)
    sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
    pitch = snd.to_pitch()
    fig = plt.figure()
    
    pitch_vals = draw_pitch(pitch)
    
    fps = 30
    frames = int(fps*snd.duration)
    X_VALS = np.linspace(0,snd.duration,frames)
    plt.xlim([snd.xmin, snd.xmax])
    ylim = plt.gca().get_ylim()
    
    xs = pitch.xs()
    interp = pd.Series(pitch_vals).interpolate("akima")
    plt.plot(xs,interp,linestyle="dashed",color="black")
    
    
    minima = xs[signal.argrelextrema(interp.values, np.less_equal,order=2)[0]]
    maxima = xs[signal.argrelextrema(interp.values, np.greater_equal,order=2)[0]]
    end = interp.ffill().iloc[-1]
    endx = xs[interp[interp==end].index[0]]
    start = interp.bfill().iloc[0]
    startx = xs[interp[interp==start].index[0]]
    
    
    if abs(ylim[0]-end)<abs(ylim[1]-end):
        minima = np.append(minima,endx)
    else:
        maxima = np.append(maxima,endx)
    if abs(ylim[0]-start)<abs(ylim[1]-start):
        minima = np.append(minima,startx)
    else:
        maxima = np.append(maxima,startx)

    #plt.plot(xs,interp.rolling(11).mean())
    #plt.plot(xs[::-1],interp[::-1].rolling(11).mean())
    
    window = 10
    points = shift(compute_intersection(xs,interp,interp.rolling(window).mean()),minima,maxima)
    points = prune(points,pitch_vals,xs)
    size_missed = window*(xs[1]-xs[0])
    #print(size_missed)
    
    pointsflat = [val for sublist in points for val in sublist]
    for i in range(0,len(pointsflat)-1,2):
        plt.fill_betweenx([ylim[0],ylim[1]],pointsflat[i],pointsflat[i+1],color="red",alpha=0.045)
    plt.vlines([i[0] for i in points],ylim[0],ylim[1],lw=0.5,color="red",alpha=0.5)
    

    l , v = plt.plot(-6, -1, 6, 1, linewidth=5, color= 'red')
    dif=X_VALS[1]-X_VALS[0]
    line_anim = animation.FuncAnimation(fig, update_line, fargs=(l,),frames=frames,interval=1/fps,blit=True)
    #fname = "vids/"+sound[sound.index("/")+1:sound.index(".")]+".gif"
    line_anim.save("munasakisanzun.gif", fps=fps);
    plt.clf()

CPU times: user 1.15 s, sys: 24.3 ms, total: 1.17 s
Wall time: 1.31 s


<Figure size 432x288 with 0 Axes>

In [9]:
import os
import time

In [15]:
now = time.time()
for n,audio in enumerate(os.listdir("audios")):
    fname = audio[:audio.index(".")]+".mp4"
    if fname in os.listdir("final"):
        print("skipped")
        continue
    makeanim("audios/"+audio)
    print(str(n)+"/"+"2000"+": "+str(time.time()-now))
    now = time.time()

skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
32/2000: 2.4762978553771973
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
51/2000: 2.377537250518799
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
s

987/2000: 2.125999689102173
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skip

<timed exec>:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


1124/2000: 2.722271680831909
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
1135/2000: 2.2154481410980225
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipp

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>